In [ ]:
from notebook_utils import *

import concurrent

In [ ]:
def pack(base_path: str, output_path: str, max_frame: int):
    os.makedirs(output_path, exist_ok=True)

    def pack_frame(frame_no: int):
        if frame_no < 2:
            return
        
        idx = "." + str(frame_no).zfill(4) + ".exr"

        # Read
        base_color_depth = ImageUtils.load_exr_image_opencv(f"{base_path}\\{GB_TYPE.BASE_COLOR_DEPTH.value}{idx}")
        mv_roughness_nov = ImageUtils.load_exr_image_opencv(f"{base_path}\\{GB_TYPE.MV_ROUGHNESS_NOV.value}{idx}")
        normal_specular = ImageUtils.load_exr_image_opencv(f"{base_path}\\{GB_TYPE.NORMAL_SPECULAR.value}{idx}")
        pretonemap_metallic = ImageUtils.load_exr_image_opencv(f"{base_path}\\{GB_TYPE.PRETONEMAP_METALLIC.value}{idx}")

        # Packed IDX start from 0000
        idx = "." + str(frame_no - 2).zfill(4) + ".exr"

        # Pack
        p_mv_metallic_roughness = np.concatenate((
            mv_roughness_nov[:, :, 0:2], # Motion Vector
            pretonemap_metallic[:, :, 3:4], # Metallic
            mv_roughness_nov[:, :, 2:3], # Roughness
        ), axis=2)
        p_nov = mv_roughness_nov[:, :, 3:4]  # Normal Occlusion Vector
        p_specular = normal_specular[:, :, 3:4]  # Specular
        p_normal_depth = np.concatenate((
            normal_specular[:, :, 0:3],  # Normal
            base_color_depth[:, :, 3:4]  # Depth
        ), axis=2)
        p_albedo = base_color_depth[:, :, 0:3] + (1 - pretonemap_metallic[:, :, 3:4]) * 0.08 * normal_specular[:, :, 3:4]

        # Save
        ImageUtils.save_exr_image_opencv(p_albedo, f"{output_path}\\Asian_Village_DemoAlbedo{idx}")
        ImageUtils.save_exr_image_opencv(base_color_depth[:, :, 0:3], f"{output_path}\\Asian_Village_DemoBaseColor{idx}")
        ImageUtils.save_exr_image_opencv(p_mv_metallic_roughness, f"{output_path}\\Asian_Village_DemoMotionVectorAndMetallicAndRoughness{idx}")
        ImageUtils.save_exr_image_opencv(p_nov, f"{output_path}\\Asian_Village_DemoNoV{idx}")
        ImageUtils.save_exr_image_opencv(p_specular, f"{output_path}\\Asian_Village_DemoSpecular{idx}")
        ImageUtils.save_exr_image_opencv(p_normal_depth, f"{output_path}\\Asian_Village_DemoWorldNormalAndSceneDepth{idx}")
        ImageUtils.save_exr_image_opencv(pretonemap_metallic[:, :, 0:3], f"{output_path}\\Asian_Village_DemoPreTonemapHDRColor{idx}")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(pack_frame, range(2, max_frame + 1)), total=max_frame - 1, desc="Packing frames"))


In [ ]:
def pack_pretonemap(base_path: str, output_path: str, max_frame: int):
    os.makedirs(output_path, exist_ok=True)

    def pack_frame(frame_no: int):
        if frame_no < 2:
            return
        idx = "." + str(frame_no).zfill(4) + ".exr"

        pretonemap_metallic = ImageUtils.load_exr_image_opencv(f"{base_path}\\{GB_TYPE.PRETONEMAP_METALLIC.value}{idx}")

        # Packed IDX start from 0000
        idx = "." + str(frame_no - 2).zfill(4) + ".exr"

        ImageUtils.save_exr_image_opencv(pretonemap_metallic[:, :, 0:3], f"{output_path}\\Asian_Village_DemoPreTonemapHDRColor{idx}")

    with concurrent.futures.ThreadPoolExecutor() as executor:
        list(tqdm(executor.map(pack_frame, range(2, max_frame + 1)), total=max_frame - 1, desc="Packing frames"))

In [ ]:
base_path = "D:\\Dev\\MinorProjDataset\\Data\\dfasr"
src_folder = "DowntownWest_1"
total_frames = 187

In [ ]:
pack_pretonemap(f"{base_path}\\{src_folder}\\1080P", f"{base_path}\\DFASR_Packed\\Nojitter-1080P", total_frames)

In [ ]:
pack(f"{base_path}\\{src_folder}\\1080P", f"{base_path}\\DFASR_Packed\\GBuffer-1080P", total_frames)

In [ ]:
pack(f"{base_path}\\{src_folder}\\540P", f"{base_path}\\DFASR_Packed\\Nojitter-540P", total_frames)

In [ ]:
pack(f"{base_path}\\{src_folder}\\360P", f"{base_path}\\DFASR_Packed\\Nojitter-360P", total_frames)

In [ ]:
pack(f"{base_path}\\{src_folder}\\270P", f"{base_path}\\DFASR_Packed\\Nojitter-270P", total_frames)